# Data Aggregation and Group Opertaions
## GrroupBy Mechanics

<figure style="float:left;">
    <center>
    <img src="http://blog.yhat.com/static/img/split-apply-combine.jpg">
    <figcaption>[ Indexing elements in a NumPy array ]</figcaption>
    </center>
</figure>

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

In [ ]:
df = DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                'key2': ['one', 'two', 'one', 'two', 'one'],
                'data1': np.random.randn(5),
                'data2': np.random.randn(5)})

df

In [ ]:
grouped = df['data1'].groupby(df['key1'])

In [ ]:
grouped

In [ ]:
grouped.mean()

In [ ]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

In [ ]:
means.unstack()

In [ ]:
## !!!
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

In [ ]:
df.groupby('key1').mean()

In [ ]:
df.groupby(['key1', 'key2']).mean()

In [ ]:
df.groupby(['key1', 'key2']).size()

### Iterating Over Groups

In [ ]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

In [ ]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

In [ ]:
pieces = dict(list(df.groupby('key1')))

In [ ]:
pieces['b']

In [ ]:
df.dtypes

In [ ]:
# axis=1 (default axis=0)
grouped = df.groupby(df.dtypes, axis=1)

In [ ]:
dict(list(grouped))

### Selecting Column or Subset of Columns

In [ ]:
df.groupby('key1')['data1']

In [ ]:
df.groupby('key1')[['data2']]

In [ ]:
df['data1'].groupby(df['key1'])

In [ ]:
df[['data2']].groupby(df['key1'])

In [ ]:
df.groupby(['key1', 'key2'])[['data2']].mean()

In [ ]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [ ]:
s_grouped

In [ ]:
s_grouped.mean()

### Grouping with Dicts and Series

In [ ]:
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

In [ ]:
people.ix[2:3, ['b', 'c']] = np.nan
people

In [ ]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 
           'd': 'blue', 'e': 'red', 'f': 'orange'}

by_columns = people.groupby(mapping, axis=1)

In [ ]:
by_columns.sum()

In [ ]:
# For series
map_series = Series(mapping)
map_series

In [ ]:
people.groupby(map_series, axis=1).count()

### Grouping with Functions

In [ ]:
people.groupby(len).sum()

In [ ]:
key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()

### Grouping by Index Levels

In [ ]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]], names=['cty', 'tenor'])

hier_df = DataFrame(np.random.randn(4, 5), columns=columns)

In [ ]:
hier_df

In [ ]:
hier_df.groupby(level='cty', axis=1).count()

## Data Aggregation

In [ ]:
df

In [ ]:
grouped = df.groupby('key1')

In [ ]:
grouped['data1'].quantile(0.9)

In [ ]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

In [ ]:
grouped.describe()

*Optimized groupby methods*

|Function name|Desctiption|
|------------|-------------|
|count||
|sum||
|mean||
|median||
|std, var||
|min, max||
|prod|Product of non-NA values|
|first, last|First and last non-NA values|

In [ ]:
tips = pd.read_csv('./tips.csv')
tips.head()

In [ ]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips[:6]

### Column-wise and Multiple Function Application

In [ ]:
grouped = tips.groupby(['sex', 'smoker'])

In [ ]:
grouped_pct = grouped['tip_pct']

In [ ]:
grouped_pct.agg('mean')

In [ ]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

In [ ]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

In [ ]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)

In [ ]:
result

In [ ]:
result['tip_pct']

In [ ]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [ ]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

In [ ]:
grouped.agg({'tip': np.max, 'size': 'sum'})

In [ ]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
             'size': 'sum'})

## Group-wise Operations and Transformations
- Reduce a one-dimensional array to a scalar value : use transform() & apply()

In [ ]:
df

In [ ]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')

k1_means

In [ ]:
pd.merge(df, k1_means, left_on='key1', right_index=True) ## !!

In [ ]:
key = ['one', 'two', 'one', 'two', 'one']

In [ ]:
people.groupby(key).mean()

In [ ]:
## transform() applies a function to each group, then places the results in the appropriate locations
people.groupby(key).transform(np.mean)

In [ ]:
def demean(arr):
    return arr - arr.mean()

In [ ]:
demeaned = people.groupby(key).transform(demean)

In [ ]:
demeaned

In [ ]:
demeaned.groupby(key).mean()

### Apply: General split-apply-combine
- The most general purpose GroupBy method
- *Splits the object* begin manipulated into pieces, *invokes the passed function* on each piece, then attempt to *concatenate* the pieces together.

In [ ]:
def top(df, n=5, columns='tip_pct'):
    return df.sort_values(by=columns)[-n::]

In [ ]:
top(tips, n=6)

In [ ]:
tips.groupby('smoker').apply(top)

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=1, columns='total_bill')

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [ ]:
result

In [ ]:
result.unstack('smoker')

In [ ]:
f = lambda x: x.describe()
grouped.apply(f)

#### Suppressing the group keys

In [ ]:
# group_keys=False
tips.groupby('smoker', group_keys=False).apply(top)

### Example: Filling Missing Values with Group-specific Values

In [ ]:
# fillna
s = Series(np.random.randn(6))

In [ ]:
s[::2] = np.nan

In [ ]:
s

In [ ]:
s.fillna(s.mean())

In [ ]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [ ]:
data

In [ ]:
# fill the NA values using the group means
fill_mean = lambda g: g.fillna(g.mean())

In [ ]:
data.groupby(group_key).apply(fill_mean)

In [ ]:
# pre-defined fill values
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])

In [ ]:
data.groupby(group_key).apply(fill_func)

## Pivot Tables and Cross-Tabulation
- *Pivot table* is a data summarization tool

In [ ]:
tips.pivot_table(index=['sex', 'smoker'])

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'],
                 columns='smoker')

In [ ]:
# margins=Ture
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'],
                 columns='smoker', margins=True)

In [ ]:
# aggfunc
tips.pivot_table('tip_pct', index=['sex', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

In [ ]:
# fill_value
tips.pivot_table('size', index=['time', 'sex', 'smoker'],
                columns='day', aggfunc='sum', fill_value=0)

*pivot_table options*

|Function name|Description|
|-------------|------------|
|values|column names or names to aggregate. By default aggregates all numeric columns|
|row|Column names or other group keys to group on the rows of the resulting pivot table|
|cols|Column names or other group keys to group on the columns of the resulting pivot table|
|aggfunc|Aggregation function or list of functions; 'mean' by default.|
|fill_value|Replace missing values in result table|
|margins|Add row/columns subtotals and grand total, False by default|

### Cross-Tabulations: Crosstab
- Special case of a pivot table that computes group frequencies

In [ ]:
from io import StringIO

data = """\
Sample    Gender    Handedness
1    Female    Right-handed
2    Male    Left-handed
3    Female    Right-handed
4    Male    Right-handed
5    Male    Left-handed
6    Male    Right-handed
7    Female    Right-handed
8    Female    Left-handed
9    Male    Right-handed
10    Female    Right-handed"""

data = pd.read_table(StringIO(data), sep='\s+')

In [ ]:
data

In [ ]:
pd.crosstab(data.Gender, data.Handedness, margins=True)

In [ ]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)